In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup

import time
import os

In [45]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [185]:
# 블로그 안에 전체목록 url을 담기
browser = webdriver.Chrome(service=service,options=chrome_options)
browser.maximize_window()
url = "https://blog.naver.com/onjeon_life/223008400649" #가져오려는 naver blog url
browser.get(url)
browser.implicitly_wait(2)
browser.switch_to.frame("mainFrame") #iframe 변환
#1~10번 목록을 돌면서 본문 읽기
urls=[]
cnt=11#초기값
while True:
    try:
        soup = BeautifulSoup(browser.page_source, 'html.parser')# 
        ## 각 1번문서의 5개의 링크 담기
        url_5 = soup.select("a.pcol2._setTop._setTopListUrl")
        url_5 = [_['href'] for _ in url_5]
        urls.append(url_5)
        #나머지 2~10 돌기
        for i in range(cnt -9 ,cnt):  
            next_tab = browser.find_element(By.CSS_SELECTOR, f"a.page.pcol2._goPageTop._param\({i}\)")
            next_tab.click()
            time.sleep(1.5)
            soup = BeautifulSoup(browser.page_source, 'html.parser')#번호가 바귈때마다 html이 바뀌므로 다시 불러옴
            url_5 = soup.select("a.pcol2._setTop._setTopListUrl")
            url_5 = [_['href'] for _ in url_5]
            urls.append(url_5)
            time.sleep(1)
        #다음페이지 클릭
        next_tab = browser.find_element(By.CSS_SELECTOR, f"a.next.pcol2._goPageTop._param\({cnt}\).aggregate_click_delegate")
        next_tab.click()
        time.sleep(1)
        cnt +=10
    except NoSuchElementException: #더이상 페이지가 없으면 탈출
        break

In [186]:
urls = [x for sublist in urls for x in sublist] #리스트안의 리스트 구조 변경

In [187]:
urls

['https://blog.naver.com/PostView.naver?blogId=onjeon_life&logNo=223008414654&categoryNo=9&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView',
 'https://blog.naver.com/PostView.naver?blogId=onjeon_life&logNo=223008414019&categoryNo=9&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView',
 'https://blog.naver.com/PostView.naver?blogId=onjeon_life&logNo=223008400649&categoryNo=9&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView',
 'https://blog.naver.com/PostView.naver?blogId=onjeon_life&logNo=223008348978&categoryNo=9&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView',
 'https://blog.naver.com/PostView.naver?blogId=onjeon_life&logNo=223008337366&categoryNo=9&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView',
 'https://blog.naver.com/PostView.naver?blogId=onjeon_life&logNo=223008327024&categoryNo=9&parentCategoryNo=0&viewDate=&curre

In [188]:
#뽑은 urls를 돌면서 본문 긁기
browser = webdriver.Chrome(service=service,options=chrome_options)
browser.maximize_window()
title = []
contents = []
for url in urls:
    browser.get(url)
    browser.implicitly_wait(2)
    soup = BeautifulSoup(browser.page_source,'html.parser')
    #페이지내 타이틀, 본문 뽑기
    texts = soup.select("span.se-fs-")
    #texts에 title, content 분리
    title.append(texts[0].text)
    contents.append([_.text for _ in texts[1:]])
    time.sleep(1.7)

In [75]:
#texts에 0은 title 나머지는 contents에 담음
title.append(texts[0].text)
contents = [_.text for _ in texts[1:]]

In [198]:
#불필요한 text제거
fin_contents=[]
for i in contents:
    result = [_ for _ in i if _ != '\u200b']
    fin_contents.append(result)
title = [_[10:] for _ in title]

[['ISTP(장인) - Conclusion(결론)',
  '결론',
  '장인만큼 대담하고 실용적인 성격 유형은 별로 없습니다. 기술적으로 숙달되어있고 유연한 대처 능력을 가지고 있는 장인은 불가능한 문제에 대한 해결책을 찾는 것에 능숙합니다. 새로운 것을 배우고 탐구하려는 이들의 성향은 개인적인 성장을 포함한 다양한 분야에서 매우 귀중하게 활용됩니다.',
  'ISTP(장인) - Introduction(소개) 포스팅으로 이동 ▼'],
 ['ISTP(장인) - Workplace Habits(직장 습관)',
  '직장 습관',
  '직장에서 장인들에게 가장 중요한 요구사항은 예측 불가능성과 흥미입니다. 이러한 성향은 개인적인 가족이나 친구들 사이의 관계에서도 어려운 요소로 작용하지만, 직장에서 만난 동료들과의 관계에서는 더욱 문제를 발생시킬 가능성이 있습니다.',
  '물론 이들이 개인적인 자유를 보장받을 수 있다면, 가장 생산적인 사람이 될 수 있습니다. 이들에게 정해진 틀을 강요하는 것은 의미가 없습니다. 장인들이 가지고 있는 재능과 특별한 관점이 인정될 때 좋은 결과로 이어질 수 있습니다.',
  '장인 직원',
  '직원으로서 장인이 가장 갈망하는 것은 개인적인 공간입니다. 여유롭고 털털한 성격을 가진 장인들은 고용주나 관리자가 어떠한 일에 몰두하도록 강요하지 않는 한 충성심을 가지고 있습니다. 엄격한 규칙, 지침, 협의는 이들의 흥미를 끌지 못하며 답답하게 느끼도록 만듭니다. 만약 이들의 행동이나 습관이 비판을 받거나 변화하도록 강요받는다면, 매우 고집스러운 모습을 보여줄 수도 있습니다.',
  '장인이 선호하는 업무는 주로 어떤 일을 수정하거나 처리해야 하는 일들입니다. 이들은 이런 업무를 효율적으로 처리하는 재능을 가지고 있으며, 실제로 움직일 수 있는 일을 선호합니다. 만약 계획을 세워야 하는 업무라면 분석가 유형의 직원을 뽑는 것이 나을 것입니다.',
  '장인 동료',
  '장인들은 종종 자신이 생각하는 것보다 동료들에게 인기가 많습니다. 조

In [212]:
import pandas as pd
df = pd.DataFrame({'title': title, 'contents': fin_contents})
df.to_csv('N_blog_crawling.csv',index=False)